In [ ]:
import enum
from collections.abc import Callable
import didppy
import util

# Dual Bound を追加で設定する関数の型
type TypeBoundExprFunc = Callable[
    [list[str], didppy.Model, list[didppy.ElementVar]],
    didppy.IntExpr
]

In [ ]:
import marimo as mo
import nbformat

# DIDP を用いたモデル

考案者の解説記事: [動的計画法ベースの数理最適化ソルバDIDPPyで最短共通超配列問題を解く](https://zenn.dev/okaduki/articles/7f9a3f3c54bc98)

In [ ]:
def boundtable_scs2(s1: str, s2: str) -> list[list[int]]:
    len1, len2 = len(s1), len(s2)

    dp = [[len1 + len2 for _ in range(len2 + 1)] for _ in range(len1 + 1)]

    for i1 in range(len1 + 1):
        dp[i1][len2] = len1 - i1
    for i2 in range(len2 + 1):
        dp[len1][i2] = len2 - i2

    for i1 in range(len1 - 1, -1, -1):
        for i2 in range(len2 - 1, -1, -1):
            if s1[i1] == s2[i2]:
                dp[i1][i2] = dp[i1 + 1][i2 + 1] + 1
            else:
                dp[i1][i2] = min(dp[i1 + 1][i2], dp[i1][i2 + 1]) + 1

    return dp

In [ ]:
def boundexpr_scs2len(
    instance: list[str],
    dpmodel: didppy.Model,
    index_vars: list[didppy.ElementVar]
) -> didppy.IntExpr:
    exprs = []
    for idx1, (s1, index_var1) in enumerate(zip(instance, index_vars)):
        for idx2, (s2, index_var2) in enumerate(zip(instance, index_vars)):
            if idx2 >= idx1:
                continue
            table_idx1_idx2 = dpmodel.add_int_table(boundtable_scs2(s1, s2))
            exprs.append(table_idx1_idx2[index_var1, index_var2])

    bound = didppy.IntExpr(0)
    for expr in exprs:
        bound = didppy.max(bound, expr)

    return bound

In [ ]:
class Model:
    def __init__(
        self,
        instance: list[str],
        extra_bounds: list[TypeBoundExprFunc] | None = None,
    ):
        chars = sorted(list(set("".join(instance))))

        dpmodel = didppy.Model(maximize=False, float_cost=False)

        index_types = [dpmodel.add_object_type(number=len(s) + 1) for s in instance]
        index_vars = [
            dpmodel.add_element_var(object_type=index_type, target=0)
            for index_type in index_types
        ]

        instance_table = dpmodel.add_element_table(
            [
                [
                    chars.index(c) for c in s
                ] + [len(chars)]
                for s in instance
            ]
        )

        dpmodel.add_base_case(
            [
                index_var == len(s)
                for s, index_var in zip(instance, index_vars)
            ]
        )

        # 文字 char に従って進む
        for id_char, char in enumerate(chars):
            condition = didppy.Condition(False)
            for sidx, index_var in enumerate(index_vars):
                condition |= (instance_table[sidx, index_var] == id_char)
            trans = didppy.Transition(
                name=f"{char}",
                cost=1 + didppy.IntExpr.state_cost(),
                effects=[
                    (
                        index_var,
                        (
                            instance_table[sidx, index_var] == id_char
                        ).if_then_else(index_var + 1, index_var),
                    )
                    for sidx, index_var in enumerate(index_vars)
                ],
                preconditions=[condition],
            )
            dpmodel.add_transition(trans)

        # 残っている文字列から 2 つを選んで SCS を取って長さが最大のものを Dual Bound とする. 
        dpmodel.add_dual_bound(boundexpr_scs2len(instance, dpmodel, index_vars))

        # 追加の Dual Bound があれば. 
        if extra_bounds:
            for bound_func in extra_bounds:
                dpmodel.add_dual_bound(bound_func(instance, dpmodel, index_vars))

        self.instance = instance
        self.dpmodel = dpmodel
        self.dpsolver = None
        self.solution = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.dpsolver = didppy.CABS(self.dpmodel, threads=12, time_limit=time_limit, quiet=(not log))
        self.solution = self.dpsolver.search()
        return self

    def to_solution(self) -> str:
        return "".join([trans.name for trans in self.solution.transitions])

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60, log: bool = False) -> str:
    model = Model(instance)
    model.solve(time_limit, log)
    return model.to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
model_01 = Model(instance_01)
solution_01 = model_01.solve().to_solution()

In [ ]:
_instance = instance_01
_model = model_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: ultckignkycosoevjuaiqfozhgmpplnbrxddbxcsvrvsnhntuqgpxzvxissbxf
str1: --t-k-gnk--------u------h-mp-----x----------nh-t-qg-xzvxis----
str2: -----i-----o----j--iqfo------lnb-x---xc-v--s----uq-p--v-issbxf
str3: ul-c-i-n-ycoso-v------oz---ppl---------------------p----------
str4: -----ig-------ev--a----z-g-----br-ddb-csvrv-n-n---g----------f

solution is feasible: True
solution is optimal: True
best bound: 62


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
model_02 = Model(instance_02)
solution_02 = model_02.solve().to_solution()

In [ ]:
_instance = instance_02
_model = model_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 98) ---
 Sol: tikojiqgfpobydeplnbrxvcazwxqgkbrfdrulhcpmqvjgtdpfuivxerbnyczodsohtvmpsubroqgxzpvnxnigplewpssbbhxdf
str1: t-k----g---------n-----------k-----u-h--m------p----x---n-------ht--------qgxz-v-x-i------s-------
str2: -i-ojiq-f-o-----lnb-x-----x-----------c---v-------------------s-------u---q---pv---i------ssb--x-f
str3: -----------------------------------ul-c-----------i-----nyc-o-so--v------o---zp------pl--p--------
str4: -i-----g------e------v-az---g-br-d------------d--------b--c---s---v-----r------vn-n-g------------f
str5: ---------p--y--pl--r----z-x--------u--cpmqv-gtd-fuiv------c--ds--------b-o------------------------
str6: ---------p-b-de------v-----------d----c---v---dpf----------z--s---

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
model_03 = Model(instance_03)
solution_03 = model_03.solve().to_solution()

In [ ]:
_instance = instance_03
_model = model_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 148) ---
  Sol: pyplrsxiboqjdgtuswevaizkgxqnfokulnpbrdcfzgpqahfjtmvdpxiuserlxgqefbnychvjeziwsgtuhaqomdkvsbncptfeomurvzdgihkoxsgzpkuqvcrnjbdsbowphzxlavknwdimgfpeysuc
str01: --------------t--------kg--n--ku-------------h---m--px------------n--h--------t---q--------------------g----x--z----v-------------x-------i------s--
str02: -------i-o-j---------i----q-fo--ln-b-----------------x------x-------c-v-----s--u--q---------p-------v---i----s-------------sb-----x----------f------
str03: ------

In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
model_04 = Model(instance_04)
solution_04 = model_04.solve().to_solution()

In [ ]:
_instance = instance_04
_model = model_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 27) ---
  Sol: bbaeddcbeacdeecbdbecabdcead
str01: ----d-cb--c---c-db-c---ce--
str02: b---dd-be---ee----e--bd----
str03: ------c--acdeec---e--b--e--
str04: --aedd-----d----d-e--bd---d
str05: --a---cbe---e-c-----ab-ce--
str06: bba----be------bd--c-b---a-
str07: bbae-----a--e--b----a-d--a-
str08: ---e----e---eecbdbe-----e--
str09: ------c---cdee--d---a-dc--d
str10: b---d----a-----bdbe-a----ad

solution is feasible: True
solution is optimal: True
best bound: 27


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
model_05 = Model(instance_05)
solution_05 = model_05.solve().to_solution()

In [ ]:
_instance = instance_05
_model = model_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 34) ---
  Sol: dabecedbaecdabcdeaecbdeabce

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
model_06 = Model(instance_06)
solution_06 = model_06.solve().to_solution()

In [ ]:
_instance = instance_06
_model = model_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TACTACGCGGTAATCGAATCGTAC
str01: -A-T--G-GG-A-T--A--CG---
str02: -A-TAC-C--T--TC----C---C
str03: --C-ACG----AAT----T-G-A-
str04: TA--A------AATC---T-GT--
str05: -A----G-G-TAA-C-AA----A-
str06: T--T-C-C--TA---G----GTA-
str07: T--T--G---TA---GA-TC-T--
str08: T-----G-GG-AA--G--T--T-C
str09: T--T-C-C---A--C-AA-C-T--
str10: T-CTA------AA-CGAA------

solution is feasible: True
solution is optimal: True
best bound: 24


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
model_07 = Model(instance_07)
solution_07 = model_07.solve().to_solution()

In [ ]:
_instance = instance_07
_model = model_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

TG---G--G-AA-GT---T-C---C-A-A---A---AG-A--T--C-AC----A--A-A---AC--AC-T---A--C-C-AGT-C-A--A--C---CT-G-A---A---GTAC--A-C--------------
str38: --GA----AG---CGT---T--A----AC-GT--GT---T-G--AG----G---A--A-A---A-G-ACA-GCT----T--AG---GA----G-A-AC----A---A---G-A-G---CT-G----G--G---
str39: A---CC--AG---CG-CA---C-T--T-C-G---G-CAG---C--G----G--CA---G---CAC---C-T-C--G----G----C-A----GCA--C-C-T--C-A---G--C--AGC-A---AC-------
str40: ATG---G--G-A-C---AA--C-T--TA---T---TC-----CTA--T-C----AT--G--T---G--C---C-A-A---GAG---G-T--T------T--TA-C---CCG---GT-G--A--C-C--A----
str41: -T-----T-GTA--G--A-T-C-TG-T----TC--TC--TA---A---ACG---A--A----C---T---T--TA-A----A--A---TC-TG-----T-GT---G----GT---T-G-T---CAC-T--C--
str42: A--ACC--A--A-C--CAA--C-T--T----TC-G--A-T--CT--CT---TG--T-AGA-TC---T----G-T----TC---T-C--T-A---A-AC--G-A---A-C--T---T---TA------------
str43: --G---G--GT----TC--TGC---C-A--G---G-CA-TAG-T--CT---T---T-----T----T---T--T---CT-G-G--CG-----GC---C-C-T-T-G-T--GTA---A---A--C-C-T-G---
str44: --G---G------C

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
model_08 = Model(instance_08)
solution_08 = model_08.solve().to_solution()

In [ ]:
_instance = instance_08
_model = model_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 44) ---
  Sol: MAPLEQSKFYTSRLNEQAHFDVAINCSFRPGVFNEKGTALHDYQ
str01: MA-L--S--Y---------------C---P-----KGT------
str02: M----QS----S-LN--A-----I-----P-V------------
str03: M-PL--S--Y------Q-HF--------R------K--------
str04: M---E----------E--H--V--N---------E----LHD--
str05: M-----S-------N----FD-AI----R---------AL----
str06: M-------F---R-N-Q-------N-S-R----N--G-------
str07: M-------FY-------AH---A----F--G-----G-----Y-
str08: M-----SKF-T-R---------------RP------------YQ
str09: M-----S-F------------VA-------GV-----TA----Q
str10: M---E-S------L-------V-------PG-FNE---------

solution is feasible: True
solution is optimal: False
best bound: 29


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
model_09 = Model(instance_09)
solution_09 = model_09.solve().to_solution()

In [ ]:
_instance = instance_09
_model = model_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

--N--------------------------TI----------------L-----P-------G----D-----F---F-------------------------W-T--------------P--S-G-------E--S-----------------V-----------------R---------V---------
str09: M--N--------------------------------T------G-----I-------------I-----D---L------F----D--------N--------------------------H----V-----DS------I------------------------PT-----IL---P---H-Q---------------------L-----A-------------T-L------------D--------------------YL-------V-------------R--------------------T------------------I-------------I---------------------------DE-------N-R---------------S--------------------V-----L--------L------------F--H---I-----------------M--G------------S--------G------
str10: M-------F--V------FL-V----------------L-----------------L---------------------P-----L----------------------V-S-----------------------S--------Q-C------V---N-----------------L-----R----T---------------R-------T-------Q----------L--P-------------------------P----------------A-------Y------------

-----F-----------------G------D------------S---------------------V--E------E----------V-------L---------------------S------E-A----R-------------Q-----------------------HL-K-----D----------GT--------------------------------
str15: M------R-----------------------YI-------V------------------S------------------P--Q--L----------------------V----LQ------V-------------G-----------------K------------------G--Q----------E---------V--E-R----------AL----Y---------L--------------T-------------P----Y------------------DY---I----D---E--------------------K----S--PI------------------Y--------------------Y---------------F-----------L----R------S--------------HL------------------------------N------------I------------Q-R---------------P---
str16: M------------P---------R---V-----P------V-----------------Y----------D--S-----P--Q-V-----S----------P---N-----------T---V------------------P--Q--A---------------R-----------L------A---T--P--------S----F---------A-------------T----P-----------T-----F---R----G-----

-----------K----G---------------N--------S--------R------S------L--------D---------------P----------------------M---R-----A------------G-----------K-----H-------D------------------V-------V---------------VIE--S------T---------K--------------------K---L-------------
str25: M------------P-----------Q-------P----L-----------K-----------------Q---SL-----------D---------------Q-------S--------------K------W------L----------------------R--------E---------A----E-------------K------H-----L-------R-----------AL-------------E------S-------L-------V---------D----------S------------------N-----L--------------E-------------E------E------------K-----L---------K---P---------------Q------------------L---------S---M------G-------------E-D---V---------------Q-----S---------------
str26: M-------F--V------FL-V----------------L-----------------L---------------------P-----L----------------------V-S-----------------------S--------Q-C------V---N-----------------L-----------------I-----T--R-------T-------Q---

-R--D------V-L----------------------------V----------R----G------------------------F-----------------G------D------------S---------------------V--E------E---------------------------F-----------LS------E-A----R-------------Q-----------------------HL-K-----D----------GT--------------------------------
str31: M-------F--V------FL-V----------------L-----------------L---------------------P-----L----------------------V-S-----------------------S--------Q-C------V-----M-------P-------L--------------F---N------------L-------------I-----T----------------T---------------T------------Q---S-----Y-----------------------T----N----------F---------------T--R------------G---------VY-----Y-P-D------K----V---------FR------S-------------------------S-------V-------L------------H---L-----------------------------------
str32: M---------------------------H-Q-I---T---V----------V-------S--------------------------G-------------P---------------T------------E---------------------V--------S-----T-----------C------

------------------------V---------------A-S-----I-----T------A-----F------K--------S------------M-----------------I----D---E----------T---W----------------D------------K-------------------------K--I---------EA-----------------------------N--T----C---------------I---------S--------------R-K-----------------HR--------N----------
str37: M---------------L-----NRIQ----------T-L----------------------------M--------------------K--------T-----AN----------------------NYE-----------------T--------------I-------E-IL-----R------------N-Y----------L-R----L----Y-I----------------I-------------------------L----------A----------R---N-----E------------------------------------E---------------------G----R----------G-------------I--------L------I-----Y-----D---D-----------------------------------N------------I--------D---------S-----V---------
str38: M----A---D---P------------A--G------TN-----G----E---E--G--------T---------------------G---C---N-------G---W------------F--------Y----------------------V-----

-------K--------------------L------------G-------------------H------------I--H-----M-----Y---------------------P--E--------G-------T-----------------E-----------Y------------V-------------L-S-----N-----F-T----------------D----------R----G----------S---R-IE-------G-----V-----------------------------------T--------H------T-V---------H--------------------
str43: M-I------------EL------R----H-----E-----V---------------------------Q-----------------G------------D------------L-------V----T--------------I-----N----V----------------V-E-------------T--P----------E----D-L--------D-----------G-----------F-------------R-------D---------------------F--I----------R------A----------------------------HL----I---------C------LA------V--D---------------------T-----------E-----------T-------------------------------------T--G---------L---------D----------I--Y-----------
str44: M-------F--V------FL-V----------------L-----------------L---------------------P-----L----------------------V-S---------------------